In [1]:
print("Hello World!")

Hello World!


In [7]:
a = !cat /etc/redhat-release
b = !cat /proc/cpuinfo | grep "model name"
c = !nproc
d = !free -h --total | grep Total
e = !gcc --version
# cloc_v = !docker run --rm aldanial/cloc --version
# angr_v = !docker-compose exec angr pip show angr | grep "Version"
# tigress_v = !docker-compose exec -t tigress obfuscate --version | grep "TIGRESS VERSION"
# ollvm_v = !docker-compose exec ollvm obfuscate --version

environment = dict(
    machine=dict(
        OS=a[0],
        CPU=b[0].split(":")[1],
        Processor_num=c[0],
        Memory=d[0].split(":")[1].split()[0]),
    tool=dict(
        gcc=dict(Version=e[0]),
        cloc=dict(Version="1.89"),
        angr=dict(Version="9.1.12332"),
        tigress=dict(Version="3.1"),
        ollvm=dict(Version="Obfuscator-LLVM clang version 4.0.1  (based on Obfuscator-LLVM 4.0.1)")
    )
)

environment

{'machine': {'OS': 'cat: /etc/redhat-release: No such file or directory',
  'CPU': ' 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz',
  'Processor_num': '8',
  'Memory': '8.0Gi'},
 'tool': {'gcc': {'Version': 'gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0'},
  'cloc': {'Version': '1.89'},
  'angr': {'Version': '9.1.12332'},
  'tigress': {'Version': '3.1'},
  'ollvm': {'Version': 'Obfuscator-LLVM clang version 4.0.1  (based on Obfuscator-LLVM 4.0.1)'}}}

In [41]:
import json
import pathlib
import pandas as pd
import statistics
pd.options.display.float_format = '{:.4g}'.format

# Useful function
def load_test_result_data(target: str) -> dict:
    target = pathlib.Path(target).joinpath("result-test")
    return {
        i.stem:[{
            "code_name": pathlib.Path(res[0]).stem,
            **res[1]}
            for res in json.loads(i.read_text())]#二次元配列のリスト内包表記
        for i in sorted(target.glob("*.json"))}


def organize_test_data(test_result_data: dict) -> list:
    return [
        {"transformation_name": k,
         "test_pass_rate": len([i["fail_cause"] for i in v if not any(i["fail_cause"])]) / len(v),
         "passed_test_num": len([i["fail_cause"] for i in v if not any(i["fail_cause"])]),
         "failed_test_num": len([i["fail_cause"] for i in v if any(i["fail_cause"])]),
         "failed_test": [i["code_name"] for i in v if any(i["fail_cause"])]
        #  } for k, v in sorted(test_result_data.items())]
         } for k, v in test_result_data.items()]


def load_similarity_data(target: str) -> dict:
    target = pathlib.Path(target).joinpath("result_similarity")
    return {
        i.stem: [{
            "code_name": pathlib.Path(res[0]).stem,
            **res[1]}
            for res in json.loads(i.read_text())]
        for i in sorted(target.glob("*.json"))}


def calc_distance_mean(similarity_data) -> list:
    return [
        {"transformation_name": k,
         "3gram_Simpson": statistics.mean([1.0 - x["simpson"] for x in v]),
         "LCSubstr": statistics.mean([1.0 - x["longest_common_substring"] for x in v])
        #  } for k, v in sorted(similarity_data.items())]
         } for k, v in similarity_data.items()]


def convert_to_dataframe(data: dict) -> pd. DataFrame:
    return pd.read_json(json.dumps(data)).set_index("transformation_name")


# Please type the name of the property to be analyzed
target_property_name = "prop1"

In [42]:
test_result_data = load_test_result_data(target_property_name)
test_pass_data = organize_test_data(test_result_data)
df_test_pass_rate = convert_to_dataframe(test_pass_data)
# df_test_pass_rate.to_markdown()
df_test_pass_rate

KeyError: "None of ['transformation_name'] are in the columns"